In [3]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import requests
import pymongo
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo

In [4]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [23]:
news_url = 'https://mars.nasa.gov/news/'
browser.visit(news_url)
html = browser.html
news_bsoup = BeautifulSoup(html, 'html.parser')

In [29]:
news_title = news_bsoup.find_all('div', class_='content_title')[0].text
news_p = news_bsoup.find_all('div', class_='article_teaser_body')[0].text
print(news_title)
print("--------------------------------------------------------------------")
print(news_p)

Mars Now
--------------------------------------------------------------------
A pair of zoomable cameras will help scientists and rover drivers with high-resolution color images.


In [30]:
jplnasa_url = 'https://www.jpl.nasa.gov'
img_url = 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16225_hires.jpg'
browser.visit(img_url)
img_html = browser.html
img_bsoup = BeautifulSoup(html, 'html.parser')

In [31]:
relative_img_path = img_bsoup.find_all('img')[3]["src"]
featured_image_url = jplnasa_url + relative_img_path
print(featured_image_url)

https://www.jpl.nasa.gov/system/missions/list_view_images/23_PIA23764-RoverNamePlateonMars-320x240.jpg


In [32]:
wthr_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(wthr_url)
wthr_html = browser.html
wthr_bsoup = BeautifulSoup(wthr_html, 'html.parser')

In [38]:
mars_weather = wthr_bsoup.find_all('p', class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text')[0].text

IndexError: list index out of range

In [39]:
fts_url = 'https://space-facts.com/mars/'
tables = pd.read_html(fts_url)

In [40]:
df = tables[2]
df.columns = ["Description", "Value"]

In [45]:
marshtml_table = df.to_html()
marshtml_table.replace('\n', '')
print(marshtml_table)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Description</th>
      <th>Value</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Surface Temperature:</td>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>7</th>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>8</th>
    

In [46]:
astro_url = 'https://astrogeology.usgs.gov'
hemispheres_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemispheres_url)
astro_html = browser.html
astro_bsoup = BeautifulSoup(astro_html, 'html.parser')

In [50]:
ahemispheres = astro_bsoup.find('div', class_='collapsible results')
marshemi = ahemispheres.find_all('div', class_='item')

hemi_img_urls = []

for i in marshemi:
    hemi = i.find('div', class_="description")
    title = hemi.h3.text
    
    hemi_link = hemi.a["href"]    
    browser.visit(astro_url + hemi_link)
    
    img_html = browser.html
    img_bsoup = BeautifulSoup(img_html, 'html.parser')
    
    img_link = img_bsoup.find('div', class_='downloads')
    img_url = img_link.find('li').a['href']

    img_dict = {}
    img_dict['title'] = title
    img_dict['img_url'] = img_url
    
    hemi_img_urls.append(img_dict)

print(hemi_img_urls)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]


In [51]:
mars_dict = {
        "news_title": news_title,
        "news_p": news_p,
        "featured_image_url": featured_image_url,
        "mars_weather": mars_weather,
        "tables": str(marshtml_table),
        "hemi_img": hemi_img_urls
    }

NameError: name 'mars_weather' is not defined